In [190]:
#from data import truncar
import pandas as pd
import numpy as np
from collections import defaultdict
import numpy as np

In [171]:
# Função manual de truncamento de casas decimais
def truncar(series, casas=4):
    fator = 10 ** casas
    return np.trunc(series * fator) / fator


In [172]:
def comparar_trilemas(df, ano1, ano2, casas=4):
    df_local = df.copy()

    # Truncar colunas especificadas
    df_local[ano1] = truncar(df_local[ano1], casas)
    df_local[ano2] = truncar(df_local[ano2], casas)

    # Valores únicos de cada ano
    set1 = set(df_local[ano1].unique())
    set2 = set(df_local[ano2].unique())

    # Interseção de valores
    valores_comuns = sorted(set1 & set2)

    # DataFrames com os estados que têm esses valores
    df_ano1 = df_local[df_local[ano1].isin(valores_comuns)][['Estado', ano1]].rename(columns={ano1: 'Valor'})
    df_ano1['Ano'] = ano1

    df_ano2 = df_local[df_local[ano2].isin(valores_comuns)][['Estado', ano2]].rename(columns={ano2: 'Valor'})
    df_ano2['Ano'] = ano2

    # Junta os dois
    df_resultado = pd.concat([df_ano1, df_ano2], ignore_index=True).drop_duplicates()

    return valores_comuns, df_resultado

In [173]:
def agrupar_estados_por_valor(df):
    """
    Recebe um DataFrame com colunas ['Estado', 'Valor', 'Ano']
    e retorna um dicionário {Valor: [lista de estados]}
    agrupando os estados pelo valor.
    """
    dicionario_valores = (
        df.groupby('Valor')['Estado']
        .apply(lambda x: sorted(x.unique()))
        .to_dict()
    )
    return dicionario_valores


In [191]:
def comparar_varios_trilemas_com_detalhes(df, colunas_trilema, casas=4):
    df_local = df.copy()

    # Truncar todas as colunas especificadas
    for col in colunas_trilema:
        df_local[col] = truncar(df_local[col], casas)

    # Estrutura para armazenar os resultados
    dicionario_total = defaultdict(lambda: {'estados': set(), 'comparacoes': set()})

    # Comparar todos os pares únicos de colunas
    for i in range(len(colunas_trilema)):
        for j in range(i + 1, len(colunas_trilema)):
            col1 = colunas_trilema[i]
            col2 = colunas_trilema[j]

            for idx, linha in df_local.iterrows():
                v1 = linha[col1]
                v2 = linha[col2]

                if v1 == v2:
                    estado = linha['Estado']
                    dicionario_total[v1]['estados'].add(estado)
                    dicionario_total[v1]['comparacoes'].add((col1, col2))

            # Também comparar valores comuns entre colunas (mesmo que em estados diferentes)
            set1 = set(df_local[col1].unique())
            set2 = set(df_local[col2].unique())
            valores_comuns = set1 & set2

            for valor in valores_comuns:
                estados1 = df_local[df_local[col1] == valor]['Estado'].unique()
                estados2 = df_local[df_local[col2] == valor]['Estado'].unique()

                dicionario_total[valor]['estados'].update(estados1)
                dicionario_total[valor]['estados'].update(estados2)
                dicionario_total[valor]['comparacoes'].add((col1, col2))

    # Converter sets em listas ordenadas
    return {
        valor: {
            'estados': sorted(info['estados']),
            'comparacoes': sorted(info['comparacoes'])
        }
        for valor, info in dicionario_total.items()
    }


In [194]:
df = pd.read_excel('Planilha Geral - Índice Trilema Brasil.xlsx', sheet_name='Trilema energético')
df_filtrado = df[['Estado','Trilema 2018','Trilema 2019','Trilema 2020','Trilema 2021','Trilema 2022']]
df_filtrado = df_filtrado.copy()
for trilema in ['Trilema 2018', 'Trilema 2019', 'Trilema 2020', 'Trilema 2021', 'Trilema 2022']:
    df_filtrado[trilema] = truncar(df_filtrado[trilema], 4)

In [186]:
colunas_trilema = ['Trilema 2018', 'Trilema 2019', 'Trilema 2020', 'Trilema 2021', 'Trilema 2022']

resultado = comparar_varios_trilemas_com_detalhes(df_filtrado, colunas_trilema)

from pprint import pprint
pprint(resultado)


{5.1814: {'comparacoes': [('Trilema 2021', 'Trilema 2022')],
          'estados': ['AMAPÁ', 'MARANHÃO']},
 5.3555: {'comparacoes': [('Trilema 2018', 'Trilema 2021')],
          'estados': ['ALAGOAS', 'AMAPÁ']},
 5.8333: {'comparacoes': [('Trilema 2018', 'Trilema 2022')],
          'estados': ['BAHIA', 'TOCANTINS']},
 5.9555: {'comparacoes': [('Trilema 2020', 'Trilema 2022')],
          'estados': ['PERNAMBUCO', 'SERGIPE']},
 6.0148: {'comparacoes': [('Trilema 2018', 'Trilema 2020')],
          'estados': ['PARAÍBA', 'RIO DE JANEIRO']},
 6.0814: {'comparacoes': [('Trilema 2019', 'Trilema 2020')],
          'estados': ['CEARÁ', 'RONDONIA']},
 6.1: {'comparacoes': [('Trilema 2020', 'Trilema 2022')],
       'estados': ['PIAUÍ', 'SÃO PAULO']},
 6.1777: {'comparacoes': [('Trilema 2020', 'Trilema 2021'),
                          ('Trilema 2020', 'Trilema 2022'),
                          ('Trilema 2021', 'Trilema 2022')],
          'estados': ['ESPÍRITO SANTO', 'RONDONIA']},
 6.3148: {'compa

In [195]:
import json

# Salvar como JSON
with open('comparacoes_trilema.json', 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)
